In [25]:
import gzip
import math
import matplotlib.pyplot as plt
import numpy
import random
import sklearn
import string
from collections import defaultdict
from gensim.models import Word2Vec
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

# load and process data

In [2]:
import gzip
import json
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [3]:
data = parse("./Video_Games.json.gz")

In [4]:
import random
data = list(data)
random.seed(0)
random.shuffle(data)

In [5]:
traindata = data[:50000]
validdata = data[50000:75000]
testdata = data[75000:100000]

In [6]:
traindata[-1]

{'overall': 1.0,
 'verified': True,
 'reviewTime': '12 14, 2016',
 'reviewerID': 'A3VK4801K2E8MG',
 'asin': 'B000WMEEBC',
 'style': {'Platform:': ' PlayStation 3'},
 'reviewerName': 'King Zulu',
 'reviewText': 'Terrible, cannot compete with the PC version...',
 'summary': 'One Star',
 'unixReviewTime': 1481673600}

In [7]:
traindata[1].keys()

dict_keys(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime'])

In [8]:
from collections import defaultdict
userset = set()
itemset = set()
averageRating = 0
ratingCount = defaultdict(int)
acc = 0
counter = 0


for d in traindata:
    userset.add(d['reviewerID'])
    itemset.add(d['asin'])
    acc += float(d['overall'])
    counter+=1
    ratingCount[d['overall']] += 1
averageRating = acc/counter

## TF-IDF

In [18]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in traindata:
    if('reviewText' not in d):
        continue 
    r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

In [19]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [20]:
words = [x[1] for x in counts[:1000]]

### Document frequency (df)

In [21]:
df = defaultdict(int)
for d in traindata:
    if('reviewText' not in d):
        continue 
    r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
    for w in set(r.split()):
        df[w] += 1

### Term frequency (tf) 

In [23]:
tf = defaultdict(int)
r = ''.join([c for c in rev['review_text'].lower() if not c in punctuation])
for w in r.split():
    # Note = rather than +=, different versions of tf could be used instead
    tf[w] = 1
    
tfidf = dict(zip(words,[tf[w] * math.log2(len(dataset) / df[w]) for w in words]))
tfidfQuery = [tf[w] * math.log2(len(dataset) / df[w]) for w in words]

NameError: name 'rev' is not defined

## TF code

In [33]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10713\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\10713\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [35]:
traindata = data[:75000]
testdata = data[75000:100000]

In [28]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [39]:
train_X , test_X = [] , [] 

for i in range(0, len(traindata)):
    words = ""
    if('reviewText' in traindata[i]):
        words  = traindata[i]['reviewText']
    review = re.sub('[^a-zA-Z]',' ',words)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)


In [40]:
for i in range(0, len(testdata)):
    words = ""
    if('reviewText' in testdata[i]):
        words  = testdata[i]['reviewText']
    review = re.sub('[^a-zA-Z]',' ',words)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [41]:
train_X[10]

'adapter work european foreign current perfectly buy wii state power supply come work foreign current burn worse case fry console purchase go replace later also price much better amazon nintendo site'

In [47]:
train_y = [ itr['overall'] for itr in traindata ] 
test_y =  [itr['overall'] for itr in testdata ]

In [42]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)

In [43]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 75000, n_features: 50153


In [44]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

In [45]:
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 25000, n_features: 50153


In [48]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

MultinomialNB()

In [49]:
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [ ]:
print(metrics.classification_report(test_y, y_pred))